In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("/home/infidea/rebirth-hjun/KRAG_2025_2/KRAG_2025/0715_kanana-1.5-8b-instruct-2505_generated_queries_train_dev_test.csv")
df.head()

,id,question,expected_output,generated_queries
0,1,"""우동이 {불을/불} 것 같아 걱정이다."" 가운데 올바른 것을 선택하고, 그 이유를...","{'answer': '""우동이 불을 것 같아 걱정이다.""가 옳다. \'붇다\'의 어...","[""부사 불 불이 아님 부사격 조사""]"
1,2,"다음 문장에서 어문 규범에 부합하지 않는 부분을 찾아 고치고, 그 이유를 설명하세요...","{'answer': '""또한 갸름한 얼굴에 초승달 같은 눈썹, 가늘고 긴 눈, 오뚝...","[""의존명사 이목구비 한글맞춤법 오똑한 오똑한 코 표준어 규정""]"
2,3,"""묘비는 1670년 현종 11년에 후손인 김려와 언양 현감이 세웠다.""를 올바르게 ...","{'answer': '""묘비는 현종 11년(1670년)에 후손인 김려와 언양 현감이...","[""묘비 건립 시기 표현 어문규정""]"
3,4,"다음 문장에서 어문 규범에 부합하지 않는 부분을 찾아 고치고, 그 이유를 설명하세요...","{'answer': '""저장할까요 또는 취소할까요?""가 옳다. 한 문장 안에 몇 개...","[""저장할까요 취소할까요 맞춤법 어문규정""]"
4,5,"""{외출 시에는/외출시에는} 에어컨을 꼭 끕시다."" 가운데 올바른 것을 선택하고, ...","{'answer': '""외출 시에는 에어컨을 꼭 끕시다.""가 옳다. 여기서의 \'시...","[""조사 생략 가능 여부 에어컨 명사구""]"


In [3]:
df['generated_queries'][0]

'["부사 불 불이 아님 부사격 조사"]'

In [4]:
cleaned_str = df['generated_queries'][0].replace('[', '').replace('"', '').replace(']', '')

In [5]:
cleaned_str

'부사 불 불이 아님 부사격 조사'

In [6]:
import torch
import pandas as pd
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

# --- 1. 기본 설정 ---
# 검색할 .txt 파일이 있는 디렉토리
DATA_PATH = "/home/infidea/rebirth-hjun/KRAG_2025/pre"
# 임베딩 모델 (다국어 지원 모델)
EMBEDDING_MODEL = "nlpai-lab/KURE-v1"
# 원본 쿼리 CSV 파일 경로
QUERY_CSV_PATH = "/home/infidea/rebirth-hjun/KRAG_2025_2/KRAG_2025/RAG/0715_kanana-1.5-8b-instruct-2505_generated_queries_train_dev_test.csv"
# 결과를 저장할 CSV 파일 경로
OUTPUT_CSV_PATH = "0715-retrieval_results.csv"

def main():
    # --- 사전 준비: 문서 로딩, 분할, 벡터 저장소 생성 (최초 1회만 실행) ---
    print("--- 2. 문서 로딩 및 분할 시작 ---")
    # 지정된 디렉토리에서 .txt 파일 로드
    loader = DirectoryLoader(
        DATA_PATH,
        glob="**/*.txt",
        loader_cls=TextLoader,
        loader_kwargs={'autodetect_encoding': True} # 인코딩 자동 감지
    )
    documents = loader.load()

    if not documents:
        print(f"오류: '{DATA_PATH}' 디렉토리에서 문서를 찾을 수 없습니다. .txt 파일이 있는지 확인하세요.")
        return

    # 텍스트를 작은 조각으로 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250, # 분할할 청크 크기
        chunk_overlap=25  # 청크 간 중첩 크기
    )
    texts = text_splitter.split_documents(documents)
    print(f"총 {len(documents)}개의 문서를 {len(texts)}개의 텍스트 조각으로 분할했습니다.")
    print("--- 문서 로딩 및 분할 완료 ---")

    print("\n--- 3. 임베딩 및 벡터 저장소 생성 시작 ---")
    # Hugging Face의 임베딩 모델 로드
    model_kwargs = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    # FAISS 벡터 저장소 생성
    vectorstore = FAISS.from_documents(texts, embeddings)
    print("--- 임베딩 및 벡터 저장소 생성 완료 ---")

    print("\n--- 4. Retriever 생성 시작 ---")
    # 벡터 저장소를 retriever로 사용 (상위 1개 문서 검색)
    retriever = vectorstore.as_retriever(search_kwargs={'k': 1})
    print("--- Retriever 생성 완료 ---")

    # --- 5. 쿼리별 문서 검색 및 결과 저장 ---
    print("\n--- 5. 질문 및 문서 검색 시작 ---")
    df = pd.read_csv(QUERY_CSV_PATH)
    
    # 쿼리 전처리 (대괄호, 따옴표 제거)
    queries = [str(q).replace('[', '').replace('"', '').replace(']', '') for q in df['generated_queries']]
    
    retrieved_contents = [] # 검색된 내용을 저장할 리스트

    # 각 쿼리에 대해 문서 검색 실행
    for i, query in enumerate(queries):
        print(f"({i+1}/{len(queries)}) 쿼리 검색 중: {query}")
        
        # retriever를 직접 사용하여 관련 문서 검색
        retrieved_docs = retriever.get_relevant_documents(query)

        if retrieved_docs:
            # 검색된 문서가 있으면 page_content를 리스트에 추가
            content = retrieved_docs[0].page_content
            retrieved_contents.append(content)
        else:
            # 검색된 문서가 없으면 빈 문자열 또는 표시 문자 추가
            retrieved_contents.append("검색된 문서 없음")

    # --- 6. 결과를 DataFrame에 추가하고 파일로 저장 ---
    print("\n--- 6. 검색 결과 취합 및 저장 시작 ---")
    # 검색된 내용을 새로운 컬럼으로 DataFrame에 추가
    df['retrieved_content'] = retrieved_contents

    # 수정된 DataFrame을 새로운 CSV 파일로 저장
    # encoding='utf-8-sig'는 Excel에서 한글이 깨지지 않게 하기 위함
    df.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"모든 검색이 완료되었으며, 결과가 '{OUTPUT_CSV_PATH}' 파일에 저장되었습니다.")
    
    # 결과 확인 (상위 5개 행 출력)
    print("\n--- 최종 결과 (상위 5개) ---")
    print(df.head())


if __name__ == '__main__':
    main()

--- 2. 문서 로딩 및 분할 시작 ---
총 1개의 문서를 214개의 텍스트 조각으로 분할했습니다.
--- 문서 로딩 및 분할 완료 ---

--- 3. 임베딩 및 벡터 저장소 생성 시작 ---


/tmp/ipykernel_2987940/2842773583.py:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


--- 임베딩 및 벡터 저장소 생성 완료 ---

--- 4. Retriever 생성 시작 ---
--- Retriever 생성 완료 ---

--- 5. 질문 및 문서 검색 시작 ---
(1/1247) 쿼리 검색 중: 부사 불 불이 아님 부사격 조사


/tmp/ipykernel_2987940/2842773583.py:77: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


(2/1247) 쿼리 검색 중: 의존명사 이목구비 한글맞춤법 오똑한 오똑한 코 표준어 규정
(3/1247) 쿼리 검색 중: 묘비 건립 시기 표현 어문규정
(4/1247) 쿼리 검색 중: 저장할까요 취소할까요 맞춤법 어문규정
(5/1247) 쿼리 검색 중: 조사 생략 가능 여부 에어컨 명사구
(6/1247) 쿼리 검색 중: 어문규정 어문규범 부합 여부 문장 수정 이유
(7/1247) 쿼리 검색 중: 의존명사 풀이 풀은 두음법칙
(8/1247) 쿼리 검색 중: 관형격 조사 '의' 띄어쓰기
(9/1247) 쿼리 검색 중: 2020 2021 기간 표기 숫자 띄어쓰기
(10/1247) 쿼리 검색 중: 관형격 조사 '의'의 중복 사용, '청나라의'에서 '의' 생략 가능
(11/1247) 쿼리 검색 중: 관형격 조사 '은' 띄어쓰기, 서술어 연결 어미 '-을 것이다' 맞춤법
(12/1247) 쿼리 검색 중: 의존명사 플래그 플래그 맞춤법 국립국어원 표준국어대사전
(13/1247) 쿼리 검색 중: 조사 '로서' '로써' 구분
(14/1247) 쿼리 검색 중: 히치하이크 두음법칙
(15/1247) 쿼리 검색 중: 의존명사 '해지다' 어간 활용 두음법칙
(16/1247) 쿼리 검색 중: 의존명사 구절 귀절 한자어 표기
(17/1247) 쿼리 검색 중: 의존명사 애드리브 외래어 표기법
(18/1247) 쿼리 검색 중: 관형격 조사 '의' 생략, 서술어 호응, 명사 형태, 띄어쓰기
(19/1247) 쿼리 검색 중: 의존명사 갭 개프 두음법칙
(20/1247) 쿼리 검색 중: 연락할게 어미 '-ㄹ게' 맞춤법
(21/1247) 쿼리 검색 중: 의존명사 튀기 트기 잡종어
(22/1247) 쿼리 검색 중: 명사 '재털이' 표준어 '재떨이'
(23/1247) 쿼리 검색 중: 조사 '이게' 부정 표현 어미 '-은데' 맞춤법
(24/1247) 쿼리 검색 중: 동사 '되다' 활용, 어간과 어미 결합, 맞춤법, '되다'의 보조동사 활용
(25/1247) 쿼리 검색 중: 일자 표기 숫자 한글 영문 